# FullControl Bambulab Template

*insert date*

#### imports

In [29]:
import fullcontrol as fc
from math import tau, sin, cos, sqrt

#### print / gcode parameters for bambulab printers

In [30]:
design_name = 'spiral'
# target = 'plot'
target = 'gcode'

printer_name='bambulab_p1'  # bambulab_x1 / bambulab_p1
material_type = 'petg'
nozzle_temp = 270
bed_temp = 70
print_speed = 1500
print_speed_percent = 100
parts_fan_percent = 40
aux_fan_percent = 0
chamber_fan_percent = 20

fan_off_layers = 3          # parts fan turns on after this layer

#### design parameters

In [31]:
EW = 0.4 # extrusion width
EH = 0.2 # extrusion height (and layer height)
initial_z = EH*0.6 # initial nozzle position is set to 0.6x the extrusion height to get a bit of 'squish' for good bed adhesion
model_height = 40
layers = int(model_height / EH)
segments_per_layer = 128

#### generate the design

In [32]:
centre = fc.Point(x=50, y=50, z=0)

steps = []
steps.append(fc.Fan(speed_percent=0))
for layer in range(layers * segments_per_layer + 1):
    z_now = initial_z + layer * EH
    if z_now == initial_z + fan_off_layers * EH:
        steps.append(fc.Fan(speed_percent=parts_fan_percent))

    layer_fraction = (layer % segments_per_layer) / segments_per_layer
    total_fraction = (int(layer / segments_per_layer) + layer_fraction) / layers
    steps.append(fc.ManualGcode(text=f'M73 P{int(total_fraction * 100)} L{layer // segments_per_layer}'))

    angle = layer_fraction * tau
    radius = 5 + 1 * cos(tau * total_fraction)
    centre.z = EH * layers * total_fraction
    steps.append(fc.polar_to_point(centre, radius, angle))
# instead of the above for-loop code, you can create the exact same design using built-in FullControl functions (uncomment the next two lines):
# rectangle_steps = fc.rectangleXY(fc.Point(x=50, y=50, z=initial_z), 50, 50)
# steps = fc.move(rectangle_steps, fc.Vector(z=EH), copy=True, copy_quantity=layers)

#### preview or generate gcode

In [33]:
# preview the design

if target == 'plot':
    # steps.append(fc.PlotAnnotation(point=steps[-1], label='finish'))
    # steps.append(fc.PlotAnnotation(point=steps[1], label='start'))

    # a default line plot to check the toolhead path of the geometry
    fc.transform(steps, 'plot', fc.PlotControls(color_type='print_sequence', style='line', line_width=0.5, zoom=0.5))

    # a plot with real heights/widths for extruded lines to preview the real 3D printed geometry
    # fc.transform(steps, 'plot', fc.PlotControls(style='tube', tube_sides=8, zoom=0.6, initialization_data={'extrusion_width': EW, 'extrusion_height': EH}))
    # fc.transform(steps, 'plot', fc.PlotControls(style='tube', tube_sides=8, zoom=0.6, initialization_data={'extrusion_width': 0.1, 'extrusion_height': 0.1}))

    # a neat preview (click the top-left button in the plot for a .png file)
    # fc.transform(steps, 'plot', fc.PlotControls(neat_for_publishing=True, zoom=0.5, initialization_data={'extrusion_width': EW, 'extrusion_height': EH}))


In [34]:
# generate and save gcode

if target == 'gcode':
    gcode_controls = fc.GcodeControls(
        printer_name=printer_name,
        save_as='fc_' + material_type + '_' + design_name,
        initialization_data={
            'model_height': model_height,
            'bed_type': "Textured PEI Plate",
            'material_type': material_type,
            'nozzle_temp': nozzle_temp,
            'bed_temp': bed_temp,
            'print_speed': print_speed,
            'print_speed_percent': print_speed_percent,
            'parts_fan_percent': parts_fan_percent,
            'aux_fan_percent': aux_fan_percent,
            'chamber_fan_percent': chamber_fan_percent,
            'extrusion_width': EW,
            'extrusion_height': EH,
            'primer': 'front_lines_then_y'})
    gcode = fc.transform(steps, 'gcode', gcode_controls)

#### please tell us what you're doing with FullControl!

- tag FullControlXYZ on social media ([twitter](https://twitter.com/FullControlXYZ), [instagram](https://www.instagram.com/fullcontrolxyz/), [linkedin](https://www.linkedin.com/in/andrew-gleadall-068587119/), [tiktok](https://www.tiktok.com/@fullcontrolxyz))
- email [info@fullcontrol.xyz](mailto:info@fullcontrol.xyz)
- post on the [subreddit](https://reddit.com/r/fullcontrol)
- post in the [github discussions or issues tabs](https://github.com/FullControlXYZ/fullcontrol/issues)

in publications, please cite the original FullControl paper and the github repo for the new python version:

- Gleadall, A. (2021). FullControl GCode Designer: open-source software for unconstrained design in additive manufacturing. Additive Manufacturing, 46, 102109. 
- Gleadall, A. and Leas, D. (2023). FullControl [electronic resource: python source code]. available at: https://github.com/FullControlXYZ/fullcontrol